In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import networkx as nx
import random
from itertools import combinations, product

# Créer un graphe G à partir de données d'entrée

def generate_random_graph(n, p):
    """Generates a random graph with `n` nodes and probability `p` of having an edge between any pair of nodes."""
    # Generate edges for the graph
    edges = []
    for i in range(n):
        for j in range(i+1, n):
            if random.random() < p:
                edges.append((i, j))
    # Create a graph from the list of edges
    G = nx.Graph(edges)
    # If the graph is not connected, re-generate it
    while not nx.is_connected(G):
        edges = []
        for i in range(n):
            for j in range(i+1, n):
                if random.random() < p:
                    edges.append((i, j))
        G = nx.Graph(edges)
    return G

# ...
def is_clique_subgraph(G, nodes):
    """Returns True if the subgraph of `G` induced by `nodes` is a clique."""
    if len(nodes) == 0:
        return False
    for u, v in combinations(nodes, 2):
        if not G.has_edge(u, v):
            return False
    return True
def is_biclique(G, nodes1, nodes2):
    """Returns True if the subgraph of `G` induced by the two sets of nodes `nodes1` and `nodes2` is a biclique,
    i.e., a complete bipartite graph."""
    # Check that both input sets are non-empty
    if len(nodes1) == 0 or len(nodes2) == 0:
        return False
    # Check that all edges between nodes in nodes1 and nodes in nodes2 exist
    for u in nodes1:
        for v in nodes2:
            if not G.has_edge(u, v):
                return False
    # Check that there are no edges between nodes in nodes1 or between nodes in nodes2
    for u, v in combinations(nodes1, 2):
        if G.has_edge(u, v):
            return False
    for u, v in combinations(nodes2, 2):
        if G.has_edge(u, v):
            return False
    return True

def algorithm(G):
    
    # Créer un ordonnancement aléatoire des sommets de G
    vertex_ordering = list(G.nodes())
    random.shuffle(vertex_ordering)
    graphs = []  # Créer une liste vide pour stocker les graphes Gi
    maximal_independent_sets_list = []

    # Pour chaque sommet vi dans l'ordonnancement
    for i, vi in enumerate(vertex_ordering):
    # Créer un nouveau graphe vide Gi
        Gi = nx.Graph()
        
        # Ajouter le sommet vi au graphe Gi
        Gi.add_node(vi)
        
    # Pour chaque sommet x dans Ni(vi)
        for x in G.neighbors(vi):
        # Ajouter le sommet x au graphe Gi
           Gi.add_node(x)
        # Ajouter une arête entre vi et x dans Gi si elle existe dans G
           if G.has_edge(vi, x):
            Gi.add_edge(vi, x)
        
    # Pour chaque sommet y dans N2i(vi)
        for y in G.neighbors(vi):
        # Ajouter le sommet y au graphe Gi
           Gi.add_node(y)
        # Ajouter une arête entre vi et y dans Gi si elle existe dans G
           if G.has_edge(vi, y):
            Gi.add_edge(vi, y)
        # Ajouter une arête entre x et y dans Gi s'il n'y en a pas dans G
           if not G.has_edge(x, y):
            Gi.add_edge(x, y)
        
    # Énumérer tous les ensembles indépendants maximaux du graphe Gi
    maximal_independent_sets = nx.maximal_independent_set(Gi)
    
    # Enregistrer les ensembles indépendants maximaux dans une liste ou un autre conteneur pour une utilisation ultérieure
    maximal_independent_sets_list.append(maximal_independent_sets) 
    graphs.append(Gi)  # Ajouter le graphe Gi à la liste des graphes
    
    # Créer un graphe vide pour stocker les cliques et les bicliques maximales de G
    maximal_cliques_and_bicliques = nx.Graph()

    # Pour chaque graphe Gi dans la liste des graphes
    for Gi in graphs:
    # Pour chaque ensemble indépendant maximal de Gi
     for maximal_independent_set in maximal_independent_sets_list:
        # Si l'ensemble indépendant maximal est une clique dans Gi
        if is_clique_subgraph(G, maximal_independent_set):
        # Ajouter la clique à maximal_cliques_and_bicliques
         maximal_cliques_and_bicliques.add_nodes_from(maximal_independent_set)
         maximal_cliques_and_bicliques.add_edges_from(combinations(maximal_independent_set, 2))
        # Si l'ensemble indépendant maximal forme une biclique dans le graphe complémentaire de Gi
        elif is_biclique(G,nx.complement(Gi), maximal_independent_set):
        # Ajouter la biclique à maximal_cliques_and_bicliques
         maximal_cliques_and_bicliques.add_nodes_from(maximal_independent_set)
         maximal_cliques_and_bicliques.add_edges_from(product(maximal_independent_set, maximal_independent_set))
    return graphs, maximal_cliques_and_bicliques





# Exécuter l'algorithme


# Conversion de la liste d'adjacence en un graphe de NetworkX
#G =  generate_random_graph(100, 0.5)
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8])
G.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (5, 6), (5, 7), (5, 8), (6, 7), (6, 8), (7, 8)])
graphs, maximal_cliques_and_bicliques = algorithm(G)


# Afficher le résultat
print("Graphes Gi :")
for Gi in graphs:
  print(Gi.nodes())
  print(Gi.edges())
print("Cliques et bicliques maximales :")
print(maximal_cliques_and_bicliques.nodes())
print(maximal_cliques_and_bicliques.edges())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Graphes Gi :
[5, 6, 7, 8]
[(5, 6), (5, 7), (5, 8), (8, 8)]
Cliques et bicliques maximales :
[7, 6, 8]
[(7, 6), (7, 8), (6, 8)]
